In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
from gradio_client import Client

# Инициализация клиента для Qwen2.5
client = Client("https://qwen-qwen2-5-72b-instruct.hf.space/")

# Список категорий
categories = [
    "BANK_SERVICE", "FOOD_GOODS", "LEASING", "LOAN",
    "NON_FOOD_GOODS", "NOT_CLASSIFIED", "REALE_STATE",
    "SERVICE", "TAX"
]

# Функция для классификации текста
def classify_text_with_qwen(text):
    """
    Классифицирует текст на основе предопределённых категорий с помощью модели Qwen2.5.
    """
    prompt = f"""
    Определи категорию для следующего текста (назначение платежа). Выбери одну из следующих категорий:
    {', '.join(categories)}.
    Текст: "{text}."
    Ответ должен быть только названием одной категории из предложенного списка.
    Вот некоторые примеры:
    Текст: "оплата договор счёт мясной деликатес ндс" Категория: FOOD_GOODS.
    Текст: "предоставление кредит договор приобретение недвижимость" Категория: BANK_SERVICE.
    Текст: "долевой инвестирование жилищный строительство" Категория: REALE_STATE.
    Обрати внимание, что категория REALE_STATE на самом деле означает Недвижимость (real state).
    Однако предсказывай именно тот класс, который указан в списке категорий.
    """
    try:
        # Отправляем запрос к модели
        result = client.predict(
            prompt,  # Вводимый текст
            [[ "None", "None" ]],  # Параметры чата (оставляем по умолчанию)
            "None",  # Дополнительный параметр
            api_name="/model_chat"  # Указываем API для Qwen2.5
        )

        # Парсим ответ модели
        predicted_category = result[1][1][1].strip().upper()

        # Проверяем, корректен ли результат
        if predicted_category in categories:
            return predicted_category
        else:
            return None  # Для некорректных ответов
    except Exception as e:
        print(f"Ошибка при обработке текста: {text}")
        print(str(e))
        return None

Loaded as API: https://qwen-qwen2-5-72b-instruct.hf.space/ ✔


In [4]:
import pandas as pd
import numpy as np

In [32]:
preprocessed_df_main = pd.read_csv("preprocessed_training.tsv", sep='\t')
preprocessed_df_main = preprocessed_df_main.drop(columns=["Unnamed: 0"])
preprocessed_df_main.head(25)

,id,amount,preprocessed_text,category
0,1,15300.0,участие конференция майкоп договор,SERVICE
1,2,40200.0,оказание услуга договор,SERVICE
2,3,1440.0,оплата порошок стиральный ariel color automat ...,NON_FOOD_GOODS
3,4,240000000.0,возврат денежный средство договор заём ндс,LOAN
4,5,1360000.0,оплата дог соглый оплата сброс загрязнять веще...,NOT_CLASSIFIED
5,6,1820000.0,оплата дог финансовый аренда акт приём-передач...,LEASING
6,7,4900.0,оплата мицеллярный вода чистый линия цветочный...,NON_FOOD_GOODS
7,8,3250.0,оплата стиральный порошок счёт,NON_FOOD_GOODS
8,9,5000.0,оплата договор счёт мясной деликатес ндс,FOOD_GOODS
9,10,1840000.0,оплата договор процентный заём ндс,LOAN


In [34]:
unlabeled_data = preprocessed_df_main['preprocessed_text'].tolist()

preprocessed_df_main['pseudo_labels'] = [None] * len(preprocessed_df_main)

for i in range(len(unlabeled_data)):
    text = unlabeled_data[i]
    if preprocessed_df_main.at[i, 'pseudo_labels'] is None:
        label = classify_text_with_qwen(text)
        preprocessed_df_main.at[i, 'pseudo_labels'] = label
    else:
        continue
    if i % 10 == 0:
        print(f"Processed {i} rows")
        if i % 100 == 0:
            preprocessed_df_main.to_csv("pseudo_labeled_data.csv", index=False)

Processed 0 rows
Processed 10 rows
Processed 20 rows
Processed 30 rows
Processed 40 rows
Processed 50 rows
Processed 60 rows
Processed 70 rows
Processed 80 rows
Ошибка при обработке текста: оказание прочий услуга
The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().
Ошибка при обработке текста: долевой инвестирование жилищный строительство
The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().
Processed 90 rows
Processed 100 rows
Ошибка при обработке текста: оплата абхазсие мандарин ндс
The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().
Processed 110 rows
Processed 120 rows
Processed 130 rows
Ошибка при обработке текста: оплата минеральный питьевой вода счёт учёт ндс
The upstream Gradio app has raised an exception but has not enabled verbo

In [35]:
preprocessed_df_main.head(100)

,id,amount,preprocessed_text,category,pseudo_labels
0,1,15300.0,участие конференция майкоп договор,SERVICE,SERVICE
1,2,40200.0,оказание услуга договор,SERVICE,SERVICE
2,3,1440.0,оплата порошок стиральный ariel color automat ...,NON_FOOD_GOODS,NON_FOOD_GOODS
3,4,240000000.0,возврат денежный средство договор заём ндс,LOAN,LOAN
4,5,1360000.0,оплата дог соглый оплата сброс загрязнять веще...,NOT_CLASSIFIED,NOT_CLASSIFIED
...,...,...,...,...,...
95,96,1830.0,оплата бумажный полотенце zewa лист рулон счёт,NON_FOOD_GOODS,NON_FOOD_GOODS
96,97,92000000.0,возврат денежный средство договор заём ндс,LOAN,LOAN
97,98,17500.0,транспортировка перевозка автомобиль старый ос...,SERVICE,SERVICE
98,99,3220.0,оплата рукав запекание econta счёт,NON_FOOD_GOODS,NON_FOOD_GOODS


### Метрика точности Qwen

In [37]:
count = 0
for i in range(len(preprocessed_df_main)):
    if preprocessed_df_main.at[i, 'pseudo_labels'] == preprocessed_df_main.at[i, 'category']:
        count += 1

print("Accuracy: ", f'{count / len(preprocessed_df_main):.2f}')

Accuracy:  0.88


In [38]:
# Сохраняем разметку в файл для последующего использования
preprocessed_df_main.to_csv("pseudo_labeled_data_training.tsv", sep='\t', index=False)

## Pseudo-labeling for preprocessed_main dataset

In [43]:
preprocessed_df_main = pd.read_csv("preprocessed_main.tsv", sep='\t')
preprocessed_df_main = preprocessed_df_main.drop(columns=["Unnamed: 0"])
preprocessed_df_main.head()

,id,amount,preprocessed_text
0,1,40500.0,тур поездка договор
1,2,32600.0,оказание услуга договор
2,3,4710.0,оплата штраф
3,4,30900.0,лечение договор
4,5,13200.0,оплата основный долг период договор оао второй...


In [44]:
preprocessed_df_main.shape

(25000, 3)

In [7]:
unlabeled_data = preprocessed_df_main['preprocessed_text'].tolist()

# preprocessed_df_main['pseudo_labels'] = [None] * len(preprocessed_df_main)

In [8]:
preprocessed_df_main = pd.read_csv("pseudo_labeled_data_main.tsv", sep='\t')
preprocessed_df_main[1000:1100]

,id,amount,preprocessed_text,pseudo_labels
1000,1001,3590000.0,строительный работа адрес москва заречный договор,SERVICE
1001,1002,2680.0,консульский сбор,SERVICE
1002,1003,87000000.0,предоставление денежный средство договор заём ндс,LOAN
1003,1004,413000000.0,предоставление денежный средство договор заём ндс,LOAN
1004,1005,970.0,оплата гель душа camay магический заклинание 4...,NON_FOOD_GOODS
...,...,...,...,...
1095,1096,1600000.0,лизинговый платёж договор ндс,LEASING
1096,1097,3170.0,налог,TAX
1097,1098,3700000.0,оплата аванс гсм счёт договор 00в ндс,NOT_CLASSIFIED
1098,1099,28800.0,покупка авиабилет артём белгород договор,SERVICE


In [31]:
for i in range(len(unlabeled_data)):
    text = unlabeled_data[i]
    if preprocessed_df_main.at[i, 'pseudo_labels'] is np.NaN:
        label = classify_text_with_qwen(text)
        preprocessed_df_main.at[i, 'pseudo_labels'] = label
    else:
        continue
    if i % 10 == 0:
        print(f"Processed {i} rows")
        if i % 100 == 0:
            preprocessed_df_main.to_csv("pseudo_labeled_data_main.tsv", sep='\t', index=False)

Processed 14070 rows


KeyboardInterrupt: 